In [1]:
import polars as pl

pl.Config.set_fmt_str_lengths(1000)

polars.config.Config

In [2]:
pepe_swaps = pl.read_parquet("pepe_swaps_4.12.23_to_4.19.23.parquet")

In [3]:
pepe_swaps.head(5)

amountOutUSD,amountInUSD,amountOut,amountIn,id,timestamp,account_id,tokenIn_symbol,tokenOut_symbol
f64,f64,f64,f64,str,str,str,str,str
1289.148284,1297.057447,6.4761e17,3.7633e27,"""0xed67bcbcab68e4387d794f63942a6728a68de4b6702f7e295ca2541b56fe6b6670000000""","""1681918703""","""0x41258d0119e4d1976009da157a7b0559dab45885""","""PEPE""","""WETH"""
3162.241888,3184.043386,1.5886e18,9.2382e27,"""0xe3b6d3dfbcb0b659be3a8bb4eb8b2223172a8bd348775c78b779e28107e38a2216010000""","""1681918703""","""0xcbcc2b2ecd195ebef03fcb7c7564e4e906485a14""","""PEPE""","""WETH"""
215.972745,218.968305,6.2662e26,1.1000e17,"""0x86db55cc9daf89b7359dab03dabbeee000e1f4df310dead4c780372d5a961918ce000000""","""1681918703""","""0x59abf65b6252db1e36108b05fb5ba76cac604353""","""WETH""","""PEPE"""
491.370852,497.655238,1.4257e27,2.5000e17,"""0x7a8d1cccf083402c1928315da84a727686ca7bc207e19330da94d55c2e57979d72010000""","""1681918703""","""0x1aa2a7d52c6f93641716eada684a7e295509ddac""","""WETH""","""PEPE"""
78.531009,79.624838,2.2785e26,4.0000e16,"""0x4bb4139e03bc81a0e6b1b792b900954dda46aac5385a67fd9138f097deabea8fec000000""","""1681918703""","""0x268d7e4fca2033c00c19e88b0fa825a032674d34""","""WETH""","""PEPE"""


In [11]:
# filter by tokenIn_symbol = PEPE
pepe_sells = pepe_swaps.filter(pl.col("tokenIn_symbol") == "PEPE")
pepe_buys = pepe_swaps.filter(pl.col("tokenOut_symbol") == "PEPE")

In [16]:
agg_sells = pepe_sells.groupby("account_id").agg(pl.sum("amountOutUSD").alias("sell_usd")).sort("sell_usd", descending=True)
agg_buys = pepe_buys.groupby("account_id").agg(pl.sum("amountOutUSD").alias("buy_usd")).sort("buy_usd", descending=True)

In [27]:
# polars join on account_id
totals = agg_sells.join(agg_buys, on="account_id", how="left").fill_null(0)

# calculate total profits with polars dataframe
totals = totals.with_columns((pl.col("sell_usd") - pl.col("buy_usd")).alias("profit_usd"))

In [30]:
totals_pd_df = totals.sort("profit_usd", descending=True).to_pandas()

In [37]:
# winners
totals_pd_df.head(20)

,account_id,sell_usd,buy_usd,profit_usd
0,0x7607709f3771502bb1fbe3aff84323369144eacf,427254.652659,0.000000,427254.652659
1,0xb58555fcba6479fced7de1485eb054943a09af7b,756956.667538,544889.822254,212066.845284
2,0x044e29b53579e93a0e0609ae6e6f86e8326c4c07,205043.104470,0.000000,205043.104470
3,0xe94e7c652b2701a87b2f73d2213399db5fc611d3,187161.239914,0.000000,187161.239914
4,0x7fe920d2366960abc38b6ab9556b7c15f75f5d35,165637.758420,0.000000,165637.758420
5,0xf27874f53038855afda1ff6049b000e598f900cb,337049.739968,172781.118069,164268.621899
6,0xcb9fbdbd60009b911ad95e35b77257ae558a14a8,147188.440817,0.000000,147188.440817
7,0x758e8229dd38cf11fa9e7c0d5f790b4ca16b3b16,241323.858883,104765.606721,136558.252162
8,0xd850632c9b314a0e6736f45e4ed677d8c0ccb3fb,135596.458291,0.000000,135596.458291
9,0x19848274b125f4f75c6d0524f5196473c023d21a,120844.076116,0.000000,120844.076116


In [38]:
# losers
totals_pd_df.tail(20)

,account_id,sell_usd,buy_usd,profit_usd
4389,0xe2ca3167b89b8cf680d63b06e8aeefc5e4ebe907,1357.374276,18433.101601,-17075.727325
4390,0x5c4a88006063f42836eb02efa17df81ef3ec216d,22905.603561,40263.519464,-17357.915903
4391,0x3a8483eafbd8ea7848070b506d301246a1ea9799,2654.568241,20578.637475,-17924.069234
4392,0xca8bccba81a5ed0639406f50c40083479c99e62e,27037.603581,45040.362176,-18002.758595
4393,0xac41a4d398ec9c92dff4a1f1a9ca8f4d4108bb8a,31538.088405,49614.907761,-18076.819356
4394,0x255ed4ae17eff35923cfde87ed6ce81c4ebf8888,5951.159640,27809.964011,-21858.804370
4395,0x87f7d434f1ec9455199bee447960cecd15dcabae,9720.083346,33710.471598,-23990.388252
4396,0x1764c139e1241ffedac3ee96291357aef501c7d7,108617.746338,132915.564380,-24297.818042
4397,0xafbd82b33e0fb070da19684be3f3db03d62edb21,1313.055393,25628.750947,-24315.695554
4398,0xba260aad190dc82a5939b8445112b126f83c13a1,305.449979,24656.489538,-24351.039559


In [39]:
# list of winners
totals_pd_df["account_id"].to_list()

['...',
 ...]